In [1]:
import gradio as gr
import torch
from torchvision import transforms
from PIL import Image
import numpy as np
from torch import nn

from skimage.color import rgb2ycbcr
from skimage.metrics import peak_signal_noise_ratio

import cv2
import numpy as np

from skimage.metrics import structural_similarity as ssim

import lpips
import os


device='cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using: {device}')

import warnings

warnings.filterwarnings("ignore")

Using: cuda


In [2]:
def calculate_psnr(gt, sr):
    scale=4
    
    y_output = rgb2ycbcr(sr)[:, :, 0] # https://in.mathworks.com/help/vision/ref/psnr.html
    y_gt = rgb2ycbcr(gt)[:, :, 0]
    
    y_output = y_output[scale:-scale, scale:-scale]
    y_gt = y_gt[scale:-scale, scale:-scale]
    
    
    # Normalize to range [0, 1]
    y_output_norm = y_output / 255.0
    y_gt_norm = y_gt / 255.0
    
    
    # Calculate PSNR
    psnr_value = peak_signal_noise_ratio(y_output_norm, y_gt_norm, data_range=1.0)
    
    return psnr_value

def calculate_ssim(img1, img2):
    # Convert images to grayscale (if needed)
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    # Compute SSIM
    ssim_value, _ = ssim(img1_gray, img2_gray, full=True)
    
    return ssim_value

def calculate_lpips(img1, img2):
    # Create the LPIPS metric
    loss_fn = lpips.LPIPS(net='alex', verbose=False)
    
    # Convert images to PyTorch tensors
    img1_tensor = lpips.im2tensor(img1)
    img2_tensor = lpips.im2tensor(img2)
    
    # Calculate LPIPS distance
    lpips_value = loss_fn(img1_tensor, img2_tensor)
    
    return lpips_value.item()

def compare_metrics(hr_img, comp_img):
    # Calculate metrics for HR vs SR
    psnr_hr_sr = calculate_psnr(hr_img, comp_img)
    ssim_hr_sr = calculate_ssim(hr_img, comp_img)
    lpips_hr_sr = calculate_lpips(hr_img, comp_img)
    
    return psnr_hr_sr, ssim_hr_sr, lpips_hr_sr
    

In [3]:
def obtainMetrics(ground_truth, to_compare):
    images = os.listdir(ground_truth)
    psnr_avg = 0
    ssim_avg = 0
    lpips_avg = 0
    for image_name in images:
        hr_image_path = os.path.join(ground_truth, image_name)
        hr_image = cv2.imread(hr_image_path)
        comp_image_path = os.path.join(to_compare, image_name)
        comp_image = cv2.imread(comp_image_path)

        if hr_image.shape != comp_image.shape:
            comp_image = cv2.resize(comp_image, (hr_image.shape[1], hr_image.shape[0]))
        
        psnr, ssim, lpips = compare_metrics(hr_image, comp_image)
        psnr_avg += psnr
        ssim_avg += ssim
        lpips_avg += lpips
        
    psnr_avg /= len(images)
    ssim_avg /= len(images)
    lpips_avg /= len(images)
    
    print(f'Average PSNR: {psnr_avg}')
    print(f'Average SSIM: {ssim_avg}')
    print(f'Average LPIPS: {lpips_avg}')

# Example usage:
# obtainMetrics('path_to_ground_truth_images', 'path_to_comparison_images')


In [4]:
obtainMetrics('./Set5/image_SRF_4/HR','./Set5/image_SRF_4/LR')

Average PSNR: 27.761990236973322
Average SSIM: 0.7805871299957905
Average LPIPS: 0.3657795786857605


In [5]:
obtainMetrics('./Set5/image_SRF_4/HR','./super_resolved_Set5_images_mobileSR/')

Average PSNR: 27.17431224843056
Average SSIM: 0.7650014058192347
Average LPIPS: 0.20001975297927857


In [6]:
obtainMetrics('./Set5/image_SRF_4/HR','./super_resolved_Set5_images_miniSRGAN')

Average PSNR: 30.134289434797903
Average SSIM: 0.8339584226734275
Average LPIPS: 0.12637634128332137


In [7]:
obtainMetrics('./Set5/image_SRF_4/HR','./super_resolved_Set5_images_miniSRResNET')

Average PSNR: 31.75703788461028
Average SSIM: 0.8829294093304441
Average LPIPS: 0.1928333267569542


In [8]:
obtainMetrics('./Set5/image_SRF_4/HR','./super_resolved_Set5_images_tinySRGAN')

Average PSNR: 30.63978584901246
Average SSIM: 0.8575719985559166
Average LPIPS: 0.1554221421480179


In [9]:
obtainMetrics('./Set5/image_SRF_4/HR','./super_resolved_Set5_images_srgan')

Average PSNR: 29.99201412809059
Average SSIM: 0.8175866713291
Average LPIPS: 0.11179292351007461


In [10]:
obtainMetrics('./Set5/image_SRF_4/HR','./super_resolved_Set5_images_EDSR')

Average PSNR: 31.778091546013663
Average SSIM: 0.8895294533114282
Average LPIPS: 0.1922380581498146


In [11]:
obtainMetrics('./Set5/image_SRF_4/HR','./super_resolved_Set5_images_FSRCNN')

Average PSNR: 30.52252834228102
Average SSIM: 0.8548452295310783
Average LPIPS: 0.2013215869665146
